In [1]:
import pandas as pd
import sqlite3
from google.colab import files

# ==========================================
# Step 1 - Upload Your CSV Files
# ==========================================
print("Upload all CSV files: competitions.csv, categories.csv, venues.csv, competitors.csv, rankings.csv, etc.")
uploaded = files.upload()

Upload all CSV files: competitions.csv, categories.csv, venues.csv, competitors.csv, rankings.csv, etc.


Saving category_details.csv to category_details.csv
Saving competition_details.csv to competition_details.csv
Saving Competitor_Rankings_Table.csv to Competitor_Rankings_Table.csv
Saving Competitors_Table.csv to Competitors_Table.csv
Saving complex_details.csv to complex_details.csv
Saving venue_details.csv to venue_details.csv


In [13]:
# ==========================================
# Step 2 - Load CSVs into Pandas DataFrames
# ==========================================
competitions_df = pd.read_csv("competition_details.csv")
categories_df = pd.read_csv("category_details.csv")
venues_df = pd.read_csv("venue_details.csv")
competitors_df = pd.read_csv("Competitors_Table.csv")
rankings_df = pd.read_csv("Competitor_Rankings_Table.csv")

# ==========================================
# Step 3 - Create SQLite Database in Memory
# ==========================================
conn = sqlite3.connect(":memory:")

# Push data into SQL tables
competitions_df.to_sql("competitions", conn, index=False, if_exists="replace")
categories_df.to_sql("categories", conn, index=False, if_exists="replace")
venues_df.to_sql("venues", conn, index=False, if_exists="replace")
competitors_df.to_sql("competitors", conn, index=False, if_exists="replace")
rankings_df.to_sql("rankings", conn, index=False, if_exists="replace")

# Helper function to run and display queries
def run_query(query):
    return pd.read_sql_query(query, conn)

# ==========================================
# Step 4 - All Step 3 Queries
# ==========================================

# --- Competitions Analysis ---
# Q1. List all competitions with category names
q1 = """
SELECT c.competition_name, cat.category_name
FROM competitions c
JOIN categories cat ON c.category_id = cat.category_id;
"""
print("Q1: Competitions with Category Names")
display(run_query(q1))

# Q2. Count competitions in each category
q2 = """
SELECT cat.category_name, COUNT(*) AS total_competitions
FROM competitions c
JOIN categories cat ON c.category_id = cat.category_id
GROUP BY cat.category_name;
"""
print("Q2: Competitions Count by Category")
display(run_query(q2))

# Q3. Find competitions of type ‘doubles’.
q3 = """
SELECT competition_id, competition_name, type, gender
FROM competitions
WHERE LOWER(type) = 'doubles';
"""
print("Q3: Competitions of type 'doubles'")
display(run_query(q3))

# Q4.Get competitions in a specific category (e.g., ITF Men).
q4 = """
SELECT competition_id, competition_name, category_id, type
FROM competitions
WHERE category_id = 'sr:category:2516';
"""
print("Q4: Competitions in ITF Men Category")
display(run_query(q4))
# Q5. Identify parent competitions and their sub-competitions.
q5 = """
SELECT parent.competition_name AS parent_competition,
       child.competition_name AS sub_competition
FROM competitions AS child
JOIN competitions AS parent
ON child.parent_id = parent.competition_id
ORDER BY parent.competition_name;
"""
print("Q5: Parent Competitions and Sub-Competitions")
display(run_query(q5))

# Q6 Analyze the distribution of competition types by category.
q6 = """
SELECT category_id, type, COUNT(*) AS total
FROM competitions
GROUP BY category_id, type
ORDER BY category_id, total DESC;
"""
print("Q6: Distribution of Competition Types by Category")
display(run_query(q6))


# Q7 List all top-level competitions (no parent).
q7 = """
SELECT competition_id, competition_name, type, gender
FROM competitions
WHERE parent_id IS NULL;
"""
print("Q7: Top-Level Competitions (No Parent)")
display(run_query(q7))

# --- Venues Analysis ---
# Q8. List all venues along with their associated complex names
q8 = """
SELECT venue_name, complex_id
FROM venues;
"""
print("Q8: Venues with Associated Complex Names")
display(run_query(q8))


# Q9. Count the number of venues in each complex
q9 = """
SELECT complex_id, COUNT(*) AS venue_count
FROM venues
GROUP BY complex_id;
"""
print("Q9: Venue Count per Complex")
display(run_query(q9))


# Q10. Get details of venues in a specific country (e.g., 'India')
q10 = """
SELECT *
FROM venues
WHERE country_name = 'India';
"""
print("Q10: Venues in India")
display(run_query(q10))


# Q11. Identify all venues with their timezones
q11 = """
SELECT venue_name, timezone
FROM venues;
"""
print("Q11: Venues with Timezones")
display(run_query(q11))


# Q12. Find complexes with more than one venue
q12= """
SELECT complex_id, COUNT(*) AS venue_count
FROM venues
GROUP BY complex_id
HAVING COUNT(*) > 1;
"""
print("Q12: Complexes with More than One Venue")
display(run_query(q12))


# Q13. List venues grouped by country
q13 = """
SELECT country_name, GROUP_CONCAT(venue_name) AS venues
FROM venues
GROUP BY country_name;
"""
print("Q13: Venues Grouped by Country")
display(run_query(q13))


# Q14. Find all venues for a specific complex (e.g., 'National Sports Complex')
q14 = """
SELECT venue_name
FROM venues
WHERE complex_id = 'sr:complex:705';
"""
print("Q14: Venues for National Sports Complex")
display(run_query(q14))


# --- Competitor Ranking Analysis ---

# Q15. Get all competitors with their rank and points
q15 = """
SELECT c.name, r.rank, r.points
FROM competitors c
JOIN rankings r ON c.competitor_id = r.competitor_id;
"""
print("Q15: Competitors with Rank and Points")
display(run_query(q15))


# Q16. Find competitors ranked in the top 5
q16 = """
SELECT c.name, r.rank, r.points
FROM competitors c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE r.rank <= 5
ORDER BY r.rank ASC;
"""
print("Q16: Competitors in Top 5")
display(run_query(q16))


# Q17. List competitors with no rank movement (stable rank)
q17 = """
SELECT c.name, r.rank, r.movement
FROM competitors c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE r.movement = 0;
"""
print("Q17: Competitors with Stable Rank")
display(run_query(q17))


# Q18. Get the total points of competitors from a specific country (e.g., 'India')
q18 = """
SELECT c.country, SUM(r.points) AS total_points
FROM competitors c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE c.country = 'India'
GROUP BY c.country;
"""
print("Q18: Total Points of Competitors from India")
display(run_query(q18))


# Q19. Count competitors per country
q19 = """
SELECT c.country, COUNT(*) AS competitor_count
FROM competitors c
JOIN rankings r ON c.competitor_id = r.competitor_id
GROUP BY c.country;
"""
print("Q19: Competitor Count per Country")
display(run_query(q19))


# Q20. Find competitors with the highest points in the current week
q20 = """
SELECT c.name, r.points
FROM competitors c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE r.points = (SELECT MAX(points) FROM rankings);
"""
print("Q20: Competitors with Highest Points This Week")
display(run_query(q20))

Q1: Competitions with Category Names


,competition_name,category_name
0,Hopman Cup,Hopman Cup
1,World Team Cup,ATP
2,World Team Cup,ATP
3,World Team Cup,ATP
4,World Team Cup,ATP
...,...,...
9870710,UTR Pancevo W01,UTR Women
9870711,UTR Pancevo W01,UTR Women
9870712,UTR Pancevo W01,UTR Women
9870713,UTR Pancevo W01,UTR Women


Q2: Competitions Count by Category


,category_name,total_competitions
0,ATP,48400
1,Billie Jean King Cup,1
2,Challenger,770884
3,Davis Cup,1
4,Exhibition,1024
5,Hopman Cup,1
6,IPTL,1
7,ITF Men,4831204
8,ITF Women,4129024
9,Juniors,256


Q3: Competitions of type 'doubles'


...

,country,total_points
0,India,18350


Q19: Competitor Count per Country


,country,competitor_count
0,Algeria,1
1,Antigua and Barbuda,1
2,Argentina,29
3,Armenia,1
4,Australia,54
...,...,...
73,Ukraine,19
74,Uruguay,3
75,Uzbekistan,1
76,Venezuela,3


Q20: Competitors with Highest Points This Week


,name,points
0,"Siniakova, Katerina",9530


In [4]:
display(competitions_df.columns)

Index(['competition_id', 'competition_name', 'parent_id', 'type', 'gender',
       'category_id'],
      dtype='object')